In [2]:
import matplotlib.pyplot as plt
import numpy as np
import spectral.io.envi as envi
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn import preprocessing
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from pathlib import Path
from PIL import Image
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout

import h5py
from skimage.transform import resize
from skimage.transform import rotate




In [3]:
datafolderpath="/../../../data/mudahmad/HSI_Human_Brain_Database_IEEE_Access/"

Loading Files from Folders and Saving in list and dicts. 

In [4]:
# patientdata=[{pat_no:filddata },{}]
patientdata=[]

def createPatientFileData():

    for path in Path(datafolderpath).iterdir():
        if not path.is_file():
            patientfiledata={}
            patientfiledata['pat_no']=path.name
            patientdata.append(patientfiledata)
            for subfolpath in Path(path).iterdir():
                if subfolpath.name == "darkReference":
                    patientfiledata['dark_ref']=subfolpath
                elif subfolpath.name == "darkReference.hdr":
                    patientfiledata['dark_ref_hdr']=subfolpath
                if subfolpath.name == "whiteReference":
                    patientfiledata['white_ref']=subfolpath
                elif subfolpath.name == "whiteReference.hdr":
                    patientfiledata['white_ref_hdr']=subfolpath
                elif subfolpath.name == "gtMap":
                    patientfiledata['gt_map']=subfolpath
                elif subfolpath.name == "gtMap.hdr":
                    patientfiledata['gt_map_hdr']=subfolpath
                elif subfolpath.name == "gtMap.jpg":
                    patientfiledata['gt_map_jpg']=subfolpath
                elif subfolpath.name == "raw":
                    patientfiledata['raw_ref']=subfolpath 
                elif subfolpath.name == "raw.hdr":
                    patientfiledata['raw_hdr']=subfolpath 
                elif subfolpath.name == "image.jpg":
                    patientfiledata['image']=subfolpath 
                elif subfolpath.name == "cropimage.jpg":
                    patientfiledata['crop_im']=subfolpath 
                else:
                    pass
                
    return patientdata
        
    
patientdata=createPatientFileData() 
            
           


<function __main__.refreshPatientData()>

PreProcessing Code for Denoising and Normalization  of the data

In [6]:
#Filter Noise from Bands from Each Pixel. 

def denoizePixalData(arr):


  pixeldata=arr

  # print(pixeldata)

  # noiseremoved= savgol_filter(pixeldata, 11, 6)
  noiseremoved = gaussian_filter(arr, sigma=5)
  # plt.figure(figsize=(20,5))
  # plt.plot(normalized, 'g')
  return noiseremoved

# Normalize Data
def normalizeData(arr):
  return  (arr-min(arr))/(max(arr)-min(arr))



Dimensionality Reduction Code. 

In [5]:
def PcaAnalysis(features):
  features=features.reshape(290*240,700)
  features = StandardScaler().fit_transform(features)


  pca = PCA(n_components=3)
  principalComponents = pca.fit_transform(features)
  return principalComponents.reshape(290,240,3)


In [ ]:
labels_dict={0:'Pixel Not Labeled', 1:'Normal Tissue', 2:'Tumour Tissue', 3:'Hypervascularized Tissue',4:"Background"}
labels_list=['Pixel Not Labeled','Normal Tissue','Tumour Tissue','Hypervascularized Tissue','Background']

    Loading File data with ENVI format, Pre-Processing and merging Spectral Data

In [7]:
spec_features_data=[]
spec_labels_data=[]
for dic in patientdata:
    
    hy=256
    wy=256
   
    norm_arr_data = np.empty((hy, wx, 700), dtype=np.float32)
    denoized_data = np.empty((hy, wx, 700), dtype=np.float32)
    for x_index in range(wx):
        for y_index in range(hy):
            denoized_data[y_index,x_index,:]=denoizePixalData(spectral_data[y_index,x_index,:])
            norm_arr_data[y_index,x_index,:]=normalizeData(denoized_data[y_index,x_index,:])
    
    norm_arr_data=norm_arr_data.reshape(290*240,700)
    print(norm_arr_data.shape)
#   pca=PcaAnalysis(norm_arr_data)
#   print("pca shape",pca.shape)
    spec_features_data.append(norm_arr_data)
    labels=gtMap_arr.reshape(290*240,)
    print(labels.shape)
    spec_labels_data.append(labels)
    
    

    
    
    
    

spec_features_data=np.array(spec_features_data)
spec_labels_data=np.array(spec_labels_data)

print(spec_features_data.shape)
print(spec_labels_data.shape)



/work/mudahmad/anaconda3/envs/vir_env/lib/python3.10/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


(290, 240, 700)
290 240
(69600, 700)
(69600,)
(290, 240, 700)
290 240
(69600, 700)
(69600,)
(290, 240, 700)
290 240
(69600, 700)
(69600,)
(290, 240, 700)
290 240
(69600, 700)
(69600,)
(290, 240, 700)
290 240
(69600, 700)
(69600,)
(290, 240, 700)
290 240
(69600, 700)
(69600,)
(290, 240, 700)
290 240
(69600, 700)
(69600,)
(290, 240, 700)
290 240
(69600, 700)
(69600,)
(290, 240, 700)
290 240
(69600, 700)
(69600,)
(290, 240, 700)
290 240
(69600, 700)
(69600,)
(290, 240, 700)
290 240
(69600, 700)
(69600,)
(290, 240, 700)
290 240
(69600, 700)
(69600,)
(290, 240, 700)
290 240
(69600, 700)
(69600,)
(290, 240, 700)
290 240
(69600, 700)
(69600,)
(290, 240, 700)
290 240
(69600, 700)
(69600,)
(290, 240, 700)
290 240
(69600, 700)
(69600,)
(290, 240, 700)
290 240
(69600, 700)
(69600,)
(290, 240, 700)
290 240
(69600, 700)
(69600,)
(290, 240, 700)
290 240
(69600, 700)
(69600,)
(290, 240, 700)
290 240
(69600, 700)
(69600,)
(290, 240, 700)
290 240
(69600, 700)
(69600,)
(290, 240, 700)
290 240
(69600, 70